In [90]:
import pandas as pd

In [91]:
file_name = "./data/Gas Sales RCI.xlsx"

In [92]:
xl = pd.ExcelFile(file_name)
for sheet in xl.sheet_names:
    df = xl.parse(sheet)
    df = df.iloc[1:]
    df.head()

In [93]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [94]:
df.head()

,Gas Sales in TJ,Unnamed: 1,Trade Group1,Unnamed: 3,Unnamed: 4,Unnamed: 5
1,2015,January,1810.515555,1067.79337,164.383104,3042.692029
2,NaN,February,1746.47472,1086.096582,163.418784,2995.990086
3,NaN,March,1368.086065,947.912266,141.858096,2457.856427
4,NaN,April,1463.376,1017.627482,153.714096,2634.717578
5,NaN,May,1152.128977,946.268698,148.87176,2247.269435


In [95]:
result_df = pd.DataFrame()

In [96]:
import calendar

In [97]:
def month_number(month: str):
    for c_index, c_month in enumerate(calendar.month_abbr):
        if (month[:3] == c_month):
            return c_index

In [98]:
#result_df["Year", "Month", "ResidentialGasSales", "CommercialGasSales", "IndustrialGasSales", "TotalGasSales", "Description"]
last_year = "2015"
for row in df.values:
    if (str(row[0]) != "nan"):
        last_year = str(row[0])
    new_row = [last_year, row[1], month_number(row[1]), row[2], row[3], row[4], row[5], "Gas Sales in TJ"]
    result_df = pd.concat([result_df, pd.DataFrame([new_row])], ignore_index=True)

In [99]:
result_df.columns = ["Year", "Month", "MonthNumber", "ResidentialGasSales", "CommercialGasSales", "IndustrialGasSales", "TotalGasSales", "Description"]

In [100]:
result_df

,Year,Month,MonthNumber,ResidentialGasSales,CommercialGasSales,IndustrialGasSales,TotalGasSales,Description
0,2015,January,1,1810.515555,1067.793370,164.383104,3042.692029,Gas Sales in TJ
1,2015,February,2,1746.474720,1086.096582,163.418784,2995.990086,Gas Sales in TJ
2,2015,March,3,1368.086065,947.912266,141.858096,2457.856427,Gas Sales in TJ
3,2015,April,4,1463.376000,1017.627482,153.714096,2634.717578,Gas Sales in TJ
4,2015,May,5,1152.128977,946.268698,148.871760,2247.269435,Gas Sales in TJ
...,...,...,...,...,...,...,...,...
114,2024,July,7,893.090499,844.761000,148.248432,1886.099931,Gas Sales in TJ
115,2024,August,8,875.354307,862.259653,152.983536,1890.597496,Gas Sales in TJ
116,2024,September,9,873.001968,862.804655,143.653920,1879.460543,Gas Sales in TJ
117,2024,October,10,892.577811,829.819803,138.157536,1860.555150,Gas Sales in TJ


In [103]:
pip install pyodbc

Note: you may need to restart the kernel to use updated packages.


In [104]:
import pyodbc

In [128]:
server = "db-at-dev-01.database.windows.net"
database = "db-at-dev-gas-sale-01"
username = "sqladmin"
password = "Password2025"

In [134]:
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)

In [135]:
cursor = cnxn.cursor()
# Insert Dataframe into SQL Server:
for index, row in result_df.iterrows():
     cursor.execute("INSERT INTO MonthlyGasSales.RCI (Year, Month, MonthNumber, ResidentialGasSales, CommercialGasSales, IndustrialGasSales, TotalGasSales, Description) values(?,?,?,?,?,?,?,?)", row.Year, row.Month, row.MonthNumber, row.ResidentialGasSales, row.CommercialGasSales, row.IndustrialGasSales, row.TotalGasSales, row.Description)
cnxn.commit()
cursor.close()